# Preparando o ambiente

In [1]:
from modules.utils import prepare_data_filesystem

prepare_data_filesystem()

[03/04/2024 20:21:11] Tentando criar o diretório data/raw.
[03/04/2024 20:21:11] O diretório data/raw já existe, nada será feito.
[03/04/2024 20:21:11] Tentando criar o diretório data/processed.
[03/04/2024 20:21:11] O diretório data/processed já existe, nada será feito.


# Programando as execuções

In [2]:
from modules.etl import bronze_step,silver_step

bronze_step(query="dengue",languague="pt")
silver_step()

[03/04/2024 20:21:12] FIST STEP (FOR EACH 1 HOUR)
[03/04/2024 20:21:12] Extracting...
[03/04/2024 20:21:12] Construindo os dados a serem enviados na requisição
[03/04/2024 20:21:12] Enviando a requisição
[03/04/2024 20:21:17] Requisição feita com sucesso!!
[03/04/2024 20:21:17] Saving response in data/raw


,author,content,description,publishedAt,source,title,url,urlToImage,load_date
0,Vitoria Lopes Gomez,"Até o final da semana passada, o Brasil já reg...","Mais de 2,3 milhões de casos prováveis da deng...",2024-04-02T08:40:00Z,"{'id': None, 'name': 'Olhardigital.com.br'}",Dengue pode piorar em pouco tempo; veja sinais...,https://olhardigital.com.br/2024/04/02/medicin...,https://olhardigital.com.br/wp-content/uploads...,2024-04-03 20:21:17.183300
1,Tamires Ferreira,"A Prefeitura de São Paulo anunciou, na sexta-f...","Data para início, no entanto, depende da chega...",2024-04-01T16:25:00Z,"{'id': None, 'name': 'Olhardigital.com.br'}",Dengue: vacinação em São Paulo começará pelas ...,https://olhardigital.com.br/2024/04/01/medicin...,https://olhardigital.com.br/wp-content/uploads...,2024-04-03 20:21:17.183300
2,Igor Nishikiori,O Corinthians estreia nesta terça-feira (2) na...,"Após quase três semanas, Timão volta a campo o...",2024-04-02T11:03:54Z,"{'id': None, 'name': 'Uol.com.br'}",Racing-URU x Corinthians: onde assistir ao jog...,https://gizmodo.uol.com.br/racing-uru-x-corint...,https://gizmodo.uol.com.br/wp-content/blogs.di...,2024-04-03 20:21:17.183300
3,https://www.facebook.com/bbcnews,"Crédito, Getty Images\r\nLegenda da foto, Agen...","À medida que os casos de dengue aumentam , as...",2024-04-01T19:34:48Z,"{'id': None, 'name': 'BBC News'}",Epidemia de dengue também está levando a aumen...,https://www.bbc.com/portuguese/articles/cqvx9w...,https://ichef.bbci.co.uk/news/1024/branded_por...,2024-04-03 20:21:17.183300
4,Redação Terra Você,"ResumoO Ministério da Saúde distribuiu 183,9 m...",Estratégia é aliada no combate ao mosquito da ...,2024-04-02T16:59:58Z,"{'id': None, 'name': 'Terra.com.br'}",Precisa abrir as janelas? Tire dúvidas sobre o...,https://www.terra.com.br/vida-e-estilo/saude/p...,https://p2.trrsf.com/image/fget/cf/1200/630/mi...,2024-04-03 20:21:17.183300
...,...,...,...,...,...,...,...,...,...
62,None,"De janeiro a 27 de março, a cidade de São Paul...","Outros 80 óbitos no município, registrados ent...",2024-04-01T14:50:28Z,"{'id': 'globo', 'name': 'Globo'}","Em uma semana, mortes por dengue na cidade de ...",https://valor.globo.com/brasil/noticia/2024/04...,https://s2-valor.glbimg.com/bEQ2snw7r87LZvdm4Q...,2024-04-03 20:21:17.183300
63,LANCE!,O Corinthians retorna aos gramados para enfren...,O Corinthians retorna aos gramados para enfren...,2024-04-02T11:08:28Z,"{'id': None, 'name': 'Lance.com.br'}",António Oliveira tem dúvida para escalação do ...,https://www.lance.com.br/corinthians/antonio...,https://p2.trrsf.com/image/fget/cf/1200/630/mi...,2024-04-03 20:21:17.183300
64,Esporte News Mundo,"O Corinthians encara o Racing, do Uruguai, nes...","O Corinthians encara o Racing, nesta terça-fei...",2024-04-02T11:06:17Z,"{'id': None, 'name': 'Terra.com.br'}",Corinthians visita Racing no Uruguai pela estr...,https://www.terra.com.br/esportes/corinthians/...,https://p2.trrsf.com/image/fget/cf/1200/630/mi...,2024-04-03 20:21:17.183300
65,Esporte News Mundo,"O Corinthians encara o Racing, do Uruguai, nes...","O Corinthians encara o Racing, nesta terça-fei...",2024-04-02T13:06:18Z,"{'id': None, 'name': 'Terra.com.br'}",Corinthians visita Racing-URU pela estreia da ...,https://www.terra.com.br/esportes/corinthians/...,https://p2.trrsf.com/image/fget/cf/1200/630/mi...,2024-04-03 20:21:17.183300
